### Transfer Learning Task

In [3]:
IMAGE_SIZE = 32
BATCH_SIZE = 64

In [28]:
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2

from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense , Conv2D , Dropout , Flatten , Activation, MaxPooling2D , GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization

from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

# 0 ~ 1사이값의 float32로 변경하는 함수
def get_preprocessed_data(images, targets, scaling=True):
    
    # 학습과 테스트 이미지 array를 0~1 사이값으로 scale 및 float32 형 변형. 
    if scaling:
        images = np.array(images/255.0, dtype=np.float32)
    else:
        images = np.array(images, dtype=np.float32)
        
    targets = np.array(targets, dtype=np.float32)
    
    return images, targets

# 0 ~ 1사이값 float32로 변경하는 함수 호출 한 뒤 OHE 적용 
def get_preprocessed_ohe(images, targets):
    images, targets = get_preprocessed_data(images, targets, scaling=False)
    # OHE 적용 
    oh_targets = to_categorical(targets)
    return images, oh_targets

# 학습/검증/테스트 데이터 세트에 전처리 및 OHE 적용한 뒤 반환 
def get_train_valid_test_set(train_images, train_targets, test_images, test_targets, validation_size=0.2, random_state=124):
    # 학습 및 테스트 데이터 세트를  0 ~ 1사이값 float32로 변경 및 OHE 적용. 
    train_images, train_oh_targets = get_preprocessed_ohe(train_images, train_targets)
    test_images, test_oh_targets = get_preprocessed_ohe(test_images, test_targets)
    
    # 학습 데이터를 검증 데이터 세트로 다시 분리
    train_train_images, validation_images, train_train_oh_targets, validation_oh_targets = train_test_split(train_images, train_oh_targets, test_size=validation_size, random_state=random_state)
    
    return (train_train_images, train_train_oh_targets), (validation_images, validation_oh_targets), (test_images, test_oh_targets) 

# 입력 image의 크기를 resize 값 만큼 증가. CIFAR100의 이미지가 32x32로 작아서 
# 마지막 feature map의 크기가 1로 되어 모델 성능이 좋지 않음. 
# 마지막 feature map의 크기를 2로 만들기 위해 resize를 64로 하여 입력 이미지 크기를 변경. 
# 단 메모리를 크게 소비하므로 64이상은 무리가 될 수 있음.
def get_resized_images(images, resize=64):
    image_cnt = images.shape[0]
#     resized된 이미지를 담아줄 배열
    resized_images = np.zeros((images.shape[0], resize, resize, 3))
    for i in range(image_cnt):
#         resize 진행
        resized_image = cv2.resize(images[i], (resize, resize))
        resized_images[i] = resized_image
    
    return resized_images

def create_model(image_size=IMAGE_SIZE, verbose=False):
    input_tensor = Input(shape=(image_size, image_size, 3))
    base_model = VGG16(input_tensor=input_tensor, include_top=False, weights='imagenet')
    base_model_output = base_model.output
    
    x = GlobalAveragePooling2D()(base_model_output)
    x = Dense(50, activation='relu')(x)
    output = Dense(100, activation='softmax', name='output')(x)
    
    model = Model(inputs=input_tensor, outputs=output)
    if verbose:
        model.summary()
    else:
        pass
    
    return model

In [29]:
IMAGE_SIZE = 32
BATCH_SIZE = 64

def train_and_evaluation(image_size=IMAGE_SIZE):
    (train_images, train_targets), (test_images, test_targets) = cifar100.load_data()
    (train_train_images, train_train_oh_targets), (validation_images, validation_oh_targets), (test_images, test_oh_targets) = \
    get_train_valid_test_set(train_images, train_targets, test_images, test_targets)
    
    print(train_train_images.shape, train_train_oh_targets.shape, validation_images.shape, validation_oh_targets.shape, test_images.shape, test_oh_targets.shape)
    
    if image_size >= 32:
        train_train_images = get_resized_images(train_train_images)
        validation_images = get_resized_images(validation_images)
        test_images = get_resized_images(test_images)
    
    else:
        pass
    
    train_generator = ImageDataGenerator(horizontal_flip=True, rescale=1./255)
    validation_generator = ImageDataGenerator(rescale=1./255)
    test_generator = ImageDataGenerator(rescale=1./255)
    
    print(train_train_images.shape)
    train_flow = train_generator.flow(train_train_images, train_train_oh_targets, batch_size=BATCH_SIZE)
    validation_flow = validation_generator.flow(validation_images, validation_oh_targets, batch_size=BATCH_SIZE, shuffle=False)
    test_flow = test_generator.flow(test_images, test_oh_targets, batch_size=BATCH_SIZE, shuffle=False)
    
    model = create_model(image_size=train_train_images.shape[1], verbose=True)
    model.compile(optimizer=Adam(), loss=CategoricalCrossentropy(), metrics=['acc'])
    
    mcp_cb = ModelCheckpoint(
        filepath='./callback_files/weights.{epoch:03d}-{val_loss:.4f}.h5', 
        monitor='val_loss', 
        save_best_only=True, 
        save_weights_only=True, 
        mode='min', 
        verbose=1)

    rlr_cb = ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.1,
        patience=2, 
        mode='min', 
        verbose=1)

    ely_cb = EarlyStopping(
        monitor='val_loss', 
        patience=4, 
        mode='min', 
        verbose=1)
    
    history = model.fit(train_flow, batch_size=BATCH_SIZE, epochs=10, validation_data=validation_flow, callbacks=[mcp_cb, rlr_cb, ely_cb])
    
    evaluation_result = model.evaluate(test_flow)
    
    return history, evaluation_result

In [30]:
import gc

# 불필요한 오브젝트를 지우는 작업
gc.collect()

3079

In [31]:
history, evaluation_result = train_and_evaluation(image_size=64)

(40000, 32, 32, 3) (40000, 100) (10000, 32, 32, 3) (10000, 100) (10000, 32, 32, 3) (10000, 100)
(40000, 64, 64, 3)
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
          

KeyboardInterrupt: 

In [ ]:
print('result:', evaluation_result)

In [ ]:
import matplotlib.pyplot as plt

def show_history(history):
    plt.figure(figsize=(6, 6))
    plt.yticks(np.arange(0, 1, 0.05))
    plt.plot(history.history['acc'], label='train')
    plt.plot(history.history['val_acc'], label='validation')
    plt.legend()
    
show_history(history)